# Estructura de agentes
- Supervisor
    - Agente Rag
    - Agente WebSearch
    - agemte respuesta

In [2]:
#from langgraph.graph import Graph
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from typing import Dict, TypedDict

In [9]:

# Creating the first analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
def analyze_question(state):
    llm = OllamaLLM(
    model="qwen2.5",
    base_url="http://192.168.0.31:11434",
    num_gpu_layers=90,       # Aceleración Metal máxima
    num_ctx=1536,            # Balance memoria/contexto
    temperature=0.5,
    )
    prompt = PromptTemplate.from_template("""
    Eres un agente que tiene que determinar si la pregunta que realiza el usuario es matematicas o es de letras. 

    Pregunta : {input}

    Analiza la pregunta. Solo responde con "matematicas" si la pregunta es de matematicas, o "letras" si la pregunta es de letras. Si no es ninguna de las dos, responde "ninguna".
  
    No des explicaciones ni detalles. Solo responde con una de las tres opciones.
    
    """)
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"decision": response, "input": state["input"]}

In [10]:
resultado = analyze_question({"input": "Cuantas vocales tiene la palabra murcielago"})
print (resultado["decision"])

letras


In [10]:
### Agente profesor matematicas
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Definimos las herramientas matemáticas
@tool
def add(a: float, b: float) -> float:
    """Suma dos números. Útil para operaciones de adición."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiplica dos números. Útil para operaciones de multiplicación."""
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide dos números. Útil para operaciones de división."""
    if b == 0:
        return "Error: No se puede dividir por cero"
    return a / b

@tool
def subtract(a: float, b: float) -> float:
    """Resta dos números. Útil para operaciones de sustracción."""
    return a - b



In [4]:
def matematicas_agent(state):
    llm = OllamaLLM(
        model="qwen2.5",
        base_url="http://localhost:11434",
        num_gpu_layers=90,
        num_ctx=1536,
        temperature=0.5,
    )
    prompt = PromptTemplate.from_template(
        """ Eres un profesor de matemáticas experto. Resuelve las preguntas utilizando tus conocimientos. Explica tu razonamiento paso a paso.

        Pregunta: {input}
        
        """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"output": response}

In [5]:
# Agente profesor de lengua
def lengua_agent(state):
    llm = OllamaLLM(
        model="qwen2.5",
        base_url="http://localhost:11434",
        num_gpu_layers=90,
        num_ctx=1536,
        temperature=0.5,
    )
    prompt = PromptTemplate.from_template(
        """ Eres un profesor de lengua experto. Resuelve las preguntas utilizando tus conocimientos. Explica tu razonamiento paso a paso.

        Pregunta: {input}
        
        """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"output": response}

In [6]:
# Agente general
def general_agent(state):
    llm = OllamaLLM(
    model="qwen2.5",
    base_url="http://localhost:11434",
    num_gpu_layers=90,       # Aceleración Metal máxima
    num_ctx=1536,            # Balance memoria/contexto
    temperature=0.5,
    )

    prompt = PromptTemplate.from_template(
        """Da una respuesta breve y concisa a la siguiente pregunta: {input}

        Si no sabes la respuesta di: "Con mis conocimientos no te puedo dar una respuesta a tu pregunta"

        """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"output": response}

In [7]:
from langgraph.graph import StateGraph, END
#You can precise the format here which could be helpfull for multimodal graphs
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
def create_graph():    
    workflow = StateGraph(AgentState)

    workflow.add_node("analyze", analyze_question)
    workflow.add_node("matematicas_agent", matematicas_agent)
    workflow.add_node("lengua_agent", lengua_agent)
    workflow.add_node("general_agent", general_agent)

    workflow.add_conditional_edges(
        "analyze",
        lambda x: x["decision"],
        {
            "matematicas": "matematicas_agent",
            "letras": "lengua_agent",
            "ninguna": "general_agent"
        }
    )

    workflow.set_entry_point("analyze")
    workflow.add_edge("matematicas_agent", END)
    workflow.add_edge("lengua_agent", END)
    workflow.add_edge("general_agent", END)

    return workflow.compile()

In [9]:
# Crea el grafo
app = create_graph()

# Ejemplo 1: Pregunta matemática
result1 = app.invoke({"input": "Calcula la suma de 5 y 3"})
print("Resultado Matemáticas:")
print(result1["output"])
print("Herramientas usadas:", result1.get("used_tools", []))

# Ejemplo 2: Pregunta de lengua
result2 = app.invoke({"input": "Analiza esta oración gramatical, la oracion es la siguiente: Quiero jugar al futbol."})
print("\nResultado Lengua:")
print(result2["output"])

# Ejemplo 3: Pregunta general
result3 = app.invoke({"input": "Que es un tornillo"})
print("\nResultado General:")
print(result3["output"])

Resultado Matemáticas:
¡Claro! Vamos a resolver esta operación paso a paso:

### Paso 1: Identificar los números a sumar
Tenemos dos números que necesitamos sumar:
- El primer número es **5**.
- El segundo número es **3**.

### Paso 2: Aplicar la regla de suma
La suma se realiza al combinar o añadir estos dos valores. En este caso, estamos sumando un número entero positivo (5) y otro número entero positivo (3).

### Paso 3: Realizar la operación aritmética
Para sumar estos números, simplemente combinamos los valores:
\[ 5 + 3 = 8 \]

### Paso 4: Verificar el resultado
Podemos verificar que nuestro cálculo es correcto si consideramos una representación visual o numérica. Por ejemplo, si tenemos 5 objetos y añadimos 3 más, en total tendremos 8 objetos.

### Conclusion
La suma de 5 y 3 es **8**.

Entonces, la respuesta final a la pregunta "Calcula la suma de 5 y 3" es:
\[ \boxed{8} \]
Herramientas usadas: []

Resultado Lengua:
Claro, voy a analizar la oración "Quiero jugar al fútbol" paso